In [4]:
import numpy as np
import pandas as pd
import imblearn                                      
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer


In [11]:
final_df = pd.read_csv('initial_preprocessing_reg.csv')

In [6]:
var_test_datapoints = round(final_df.shape[0] * 0.1)
df_nans = final_df[final_df['DAS28-CRP 3m'].isnull()]           ### making a new dataframe for unknown values in the target variable
final_df = final_df[final_df['DAS28-CRP 3m'].notnull()]         ### Removing the unknown category       
         

In [7]:
df_nans_x = df_nans.drop('DAS28-CRP 3m',1)
df_nans_y = df_nans['DAS28-CRP 3m']

<ipython-input-7-39b7167b4040>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_nans_x = df_nans.drop('DAS28-CRP 3m',1)


In [8]:
rs = 1337
X = final_df.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y set
y = final_df['DAS28-CRP 3m']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=var_test_datapoints,random_state = rs)
combined_train_df_x = pd.concat([X_train, df_nans_x])

<ipython-input-8-d89d024094ce>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = final_df.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y set


In [14]:
from sklearn.impute import KNNImputer                         ### Imputing the null values using the X(features) values for both known and unknown values for the target variable
imputer = KNNImputer(n_neighbors=15)
fit_train = imputer.fit(combined_train_df_x)                  # (shape = 250,77) does not include target variable
imput_train = fit_train.transform(combined_train_df_x)     
c = list(combined_train_df_x.columns)
imput_train_df = pd.DataFrame(imput_train, columns = c)

imput_test = fit_train.transform(X_test)           ### Imputing the test dataset X(features) using the imputer trained from the train dataset
c = list(X_test.columns)
imput_test_df = pd.DataFrame(imput_test, columns = c)

### combining X and y for the training set
combined_train_df_y = pd.concat([Y_train, df_nans_y],ignore_index=True)
imput_train_df['DAS28-CRP 3m'] = combined_train_df_y

fit_train_x_y = imputer.fit(imput_train_df)        ### Imputing the unknown values in the target variable using the combined X and y of the training set
imput_train_2 = fit_train_x_y.transform(imput_train_df)
c = list(imput_train_df.columns)
imput_train_df_x_y = pd.DataFrame(imput_train_2, columns = c)

In [15]:
X_unknown = imput_train_df_x_y.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y and including the unknown datapoints
y_unknown = imput_train_df_x_y['DAS28-CRP 3m']

<ipython-input-15-5794b32b7c14>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = imput_train_df_x_y.drop('DAS28-CRP 3m',1)                            ### Dividing the dataframe into X and y and including the unknown datapoints


In [17]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in X_unknown.columns:
    if i not in labels:
        mean = X_unknown[i].mean()
        std = X_unknown[i].std()
        X_unknown[i] = (X_unknown[i] - mean) / std
        imput_test_df[i] = (imput_test_df[i] - mean) / std

In [18]:
### Remove "hxhtn" feature here if the code is not running
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    X_unknown = X_unknown.drop(i,1)
    imput_test_df = imput_test_df.drop(i,1)

<ipython-input-18-5fd5511c3b31>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = X_unknown.drop(i,1)
<ipython-input-18-5fd5511c3b31>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  imput_test_df = imput_test_df.drop(i,1)


In [10]:
X_unknown['y'] = y_unknown
X_unknown.to_csv('train_method2_rg.csv', index = False)

In [16]:
Y_test1 = Y_test.reset_index()
Y_test1 = Y_test1.drop(columns=['index'])
imput_test_df['y'] = Y_test1
imput_test_df.to_csv('test_method2_rg.csv', index = False)